# sorting with instances of the `IGrouping<TKey,TElement>` interface

The `IGrouping<TKey,TElement>` interface [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.igrouping-2?view=net-8.0)] returns when the `Enumerable.GroupBy` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.groupby?view=net-8.0)] method of LINQ is called. We can see this by starting with some ‘groupable’ `data`:

In [1]:
IReadOnlyCollection<(string groupName, string displayText)> data =
    new []
    {
        ("g2", "w-thing"),
        ("g1", "g-thing"),
        ("g3", "q-thing"),
        ("g1", "a-thing"),
        ("g2", "x-thing"),
        ("g3", "z-thing"),
        ("g1", "k-thing"),
        ("g2", "p-thing"),
        ("g3", "r-thing"),
    };

Calling `GroupBy` followed by `ToArray` [📖 [docs](https://learn.microsoft.com/en-us/dotnet/api/system.linq.enumerable.toarray?view=net-8.0)] allows us to set the variable `groups` as a read-only collection:

In [2]:
IReadOnlyCollection<IGrouping<string, (string groupName, string displayText)>> groups =
    data.GroupBy(tuple => tuple.groupName).ToArray();

groups

index value 0 [ (g2, w-thing), (g2, x-thing), (g2, p-thing) ] Key g2 (values) index value 0 (g2, w-thing) Item1 g2 Item2 w-thing 1 (g2, x-thing) Item1 g2 Item2 x-thing 2 (g2, p-thing) Item1 g2 Item2 p-thing 1 [ (g1, g-thing), (g1, a-thing), (g1, k-thing) ] Key g1 (values) index value 0 (g1, g-thing) Item1 g1 Item2 g-thing 1 (g1, a-thing) Item1 g1 Item2 a-thing 2 (g1, k-thing) Item1 g1 Item2 k-thing 2 [ (g3, q-thing), (g3, z-thing), (g3, r-thing) ] Key g3 (values) index value 0 (g3, q-thing) Item1 g3 Item2 q-thing 1 (g3, z-thing) Item1 g3 Item2 z-thing 2 (g3, r-thing) Item1 g3 Item2 r-thing

Our `groups` variable is a read-only collection of `IGrouping<TKey,TElement>` where `TKey` is the type of our `GroupBy` expression and `TElement` is our tuple type. We can explicitly see the grouping key with the following LINQ projection:

In [3]:
groups.Select(group => group.Key)

[ g2, g1, g3 ]

## sorting by group “fluently”

The danger of sorting by `displayText` within each group “fluently” looms when we do not not intend to lose the grouping. The following approach loses the grouping:

In [4]:
var sorted = groups.Select(group => group.OrderBy(tuple => tuple.displayText));

sorted

index value 0 index value 0 (g2, p-thing) Item1 g2 Item2 p-thing 1 (g2, w-thing) Item1 g2 Item2 w-thing 2 (g2, x-thing) Item1 g2 Item2 x-thing 1 index value 0 (g1, a-thing) Item1 g1 Item2 a-thing 1 (g1, g-thing) Item1 g1 Item2 g-thing 2 (g1, k-thing) Item1 g1 Item2 k-thing 2 index value 0 (g3, q-thing) Item1 g3 Item2 q-thing 1 (g3, r-thing) Item1 g3 Item2 r-thing 2 (g3, z-thing) Item1 g3 Item2 z-thing

In [5]:
sorted.GetType()

System.Linq.Enumerable+SelectArrayIterator<System.Linq.IGrouping<System.String,System.ValueTuple<System.String,System.String>>,System.Linq.IOrderedEnumerable<System.ValueTuple<System.String,System.String>>>

## sorting each group before grouping

The most successful way to sort within each group was described in 2018 by Raymond Chen in “[Taking advantage of the ordering guarantees of the LINQ GroupBy method](https://devblogs.microsoft.com/oldnewthing/20181129-00/?p=100355).” The secret lies in sorting _before_ grouping, taking advantage of document order:

In [6]:
IReadOnlyCollection<IGrouping<string, (string groupName, string displayText)>> sortedGroups =
    data
    .OrderBy(i => i.groupName)
    .ThenBy(i => i.displayText)
    .GroupBy(tuple => tuple.groupName).ToArray();

sortedGroups

index value 0 [ (g1, a-thing), (g1, g-thing), (g1, k-thing) ] Key g1 (values) index value 0 (g1, a-thing) Item1 g1 Item2 a-thing 1 (g1, g-thing) Item1 g1 Item2 g-thing 2 (g1, k-thing) Item1 g1 Item2 k-thing 1 [ (g2, p-thing), (g2, w-thing), (g2, x-thing) ] Key g2 (values) index value 0 (g2, p-thing) Item1 g2 Item2 p-thing 1 (g2, w-thing) Item1 g2 Item2 w-thing 2 (g2, x-thing) Item1 g2 Item2 x-thing 2 [ (g3, q-thing), (g3, r-thing), (g3, z-thing) ] Key g3 (values) index value 0 (g3, q-thing) Item1 g3 Item2 q-thing 1 (g3, r-thing) Item1 g3 Item2 r-thing 2 (g3, z-thing) Item1 g3 Item2 z-thing

## grouping by an anonymous object

In [5]:
IReadOnlyCollection<(string groupName, string subGroup, string displayText)> complexData =
    new []
    {
        ("g2", "sg-one", "w-thing"),
        ("g1", "sg-one", "g-thing"),
        ("g3", "sg-one", "q-thing"),
        ("g1", "sg-two", "a-thing"),
        ("g2", "sg-two", "x-thing"),
        ("g3", "sg-one", "z-thing"),
        ("g1", "sg-one", "k-thing"),
        ("g2", "sg-two", "p-thing"),
        ("g3", "sg-one", "r-thing"),
    };

In [6]:
complexData.GroupBy(i => new {i.groupName, i.subGroup})

index value 0 [ (g2, sg-one, w-thing) ] Key { groupName = g2, subGroup = sg-one } groupName g2 subGroup sg-one (values) index value 0 (g2, sg-one, w-thing) Item1 g2 Item2 sg-one Item3 w-thing 1 [ (g1, sg-one, g-thing), (g1, sg-one, k-thing) ] Key { groupName = g1, subGroup = sg-one } groupName g1 subGroup sg-one (values) index value 0 (g1, sg-one, g-thing) Item1 g1 Item2 sg-one Item3 g-thing 1 (g1, sg-one, k-thing) Item1 g1 Item2 sg-one Item3 k-thing 2 [ (g3, sg-one, q-thing), (g3, sg-one, z-thing), (g3, sg-one, r-thing) ] Key { groupName = g3, subGroup = sg-one } groupName g3 subGroup sg-one (values) index value 0 (g3, sg-one, q-thing) Item1 g3 Item2 sg-one Item3 q-thing 1 (g3, sg-one, z-thing) Item1 g3 Item2 sg-one Item3 z-thing 2 (g3, sg-one, r-thing) Item1 g3 Item2 sg-one Item3 r-thing 3 [ (g1, sg-two, a-thing) ] Key { groupName = g1, subGroup = sg-two } groupName g1 subGroup sg-two (values) index value 0 (g1, sg-two, a-thing) Item1 g1 Item2 sg-two Item3 a-thing 4 [ (g2, sg-two, x-thing), (g2, sg-two, p-thing) ] Key { groupName = g2, subGroup = sg-two } groupName g2 subGroup sg-two (values) index value 0 (g2, sg-two, x-thing) Item1 g2 Item2 sg-two Item3 x-thing 1 (g2, sg-two, p-thing) Item1 g2 Item2 sg-two Item3 p-thing

Changing the order of properties on the anonymous object has no effect:

In [7]:
complexData.GroupBy(i => new {i.subGroup, i.groupName})

index value 0 [ (g2, sg-one, w-thing) ] Key { subGroup = sg-one, groupName = g2 } subGroup sg-one groupName g2 (values) index value 0 (g2, sg-one, w-thing) Item1 g2 Item2 sg-one Item3 w-thing 1 [ (g1, sg-one, g-thing), (g1, sg-one, k-thing) ] Key { subGroup = sg-one, groupName = g1 } subGroup sg-one groupName g1 (values) index value 0 (g1, sg-one, g-thing) Item1 g1 Item2 sg-one Item3 g-thing 1 (g1, sg-one, k-thing) Item1 g1 Item2 sg-one Item3 k-thing 2 [ (g3, sg-one, q-thing), (g3, sg-one, z-thing), (g3, sg-one, r-thing) ] Key { subGroup = sg-one, groupName = g3 } subGroup sg-one groupName g3 (values) index value 0 (g3, sg-one, q-thing) Item1 g3 Item2 sg-one Item3 q-thing 1 (g3, sg-one, z-thing) Item1 g3 Item2 sg-one Item3 z-thing 2 (g3, sg-one, r-thing) Item1 g3 Item2 sg-one Item3 r-thing 3 [ (g1, sg-two, a-thing) ] Key { subGroup = sg-two, groupName = g1 } subGroup sg-two groupName g1 (values) index value 0 (g1, sg-two, a-thing) Item1 g1 Item2 sg-two Item3 a-thing 4 [ (g2, sg-two, x-thing), (g2, sg-two, p-thing) ] Key { subGroup = sg-two, groupName = g2 } subGroup sg-two groupName g2 (values) index value 0 (g2, sg-two, x-thing) Item1 g2 Item2 sg-two Item3 x-thing 1 (g2, sg-two, p-thing) Item1 g2 Item2 sg-two Item3 p-thing

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼